## Transformations

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from math import sin,cos
from collections import Counter
from PIL import Image
%matplotlib inline

# 1a

In [ ]:
import numpy as np
from numpy import sin, cos

def ICV_rotate(deg=None):
    # Function to generate a rotation matrix given an angle in degrees
    if deg is None:
        rad = np.radians(int(input("Enter rotation angle in degrees (+/-):")))
    else:
        rad = np.radians(deg)
    rotation_matrix = np.array([[cos(rad), -sin(rad)], [sin(rad), cos(rad)]])
    return rotation_matrix

def ICV_skew(deg=None):
    # Function to generate a skew matrix given an angle in degrees
    if deg is None:
        rad = np.radians(int(input("Enter skewing angle in degrees (+/-):")))
    else:
        rad = np.radians(deg)
    skew_matrix = np.array([[1, 0], [1 / np.tan(rad), 1]])
    return skew_matrix

def create_canvas(h, w, c, transform_matrix, offset):
    # Function to create a canvas based on the transformation matrix and offset
    corner_points = [
        [0 - offset[1], 0 - offset[1], h - offset[1], h - offset[1]],
        [0 - offset[0], w - offset[0], w - offset[0], 0 - offset[0]]
    ]

    # Apply the transformation matrix to corner points
    transformed_corners = np.matmul(transform_matrix, corner_points)

    # Calculate the dimensions of the canvas based on the transformed corner points
    max_y, max_x = np.max(np.abs(transformed_corners), axis=1)
    min_y, min_x = np.min(transformed_corners, axis=1)

    max_x, max_y = int(2 * max_x), int(2 * max_y)
    canvas = np.zeros((max_y, max_x, c), np.int32)

    return canvas, (max_x, max_y), (min_x, min_y)

def ICV_transform(img, transform_matrices):
    h, w, c = img.shape
    offset = (w // 2, h // 2)

    # Create a transform_matrix from the dot product of transformation matrices
    transform_matrix = np.dot(*transform_matrices) if len(transform_matrices) > 1 else transform_matrices[0]

    # Generate a new canvas with offset points as origin
    new_canvas, (max_x, max_y), (min_x, min_y) = create_canvas(h, w, c, transform_matrix, offset)

    for i in range(h):
        for j in range(w):
            # Apply the inverse transformation to map the pixel back to the original position
            new_i, new_j = np.matmul(transform_matrix, [[i - offset[1]], [j - offset[0]]])

            # Adjust coordinates based on minimum values (handling negative values)
            if min_y < 0:
                new_i += abs(min_y)
            if min_x < 0:
                new_j += abs(min_x)

            # Check if the transformed pixel is within the canvas boundaries
            if 0 <= new_i < max_y and 0 <= new_j < max_x:
                new_canvas[int(new_i), int(new_j)] = img[i, j]

    return new_canvas


# 1b

In [ ]:
fig, ax = plt.subplots()
font_properties = {
    'family': 'sans-serif',
    'size': 72,
}

ax.text(0.5, 0.5, "RUTHWIK", **font_properties, ha='center', va='center')
plt.axis('off')
plt.savefig('ruthwik.jpg') 
plt.title("Original image")

In [ ]:
path = './ruthwik.jpg'
img_bgr = cv2.imread(path, 1) 

In [ ]:
output = ICV_transform(img_bgr, [ICV_rotate(30)])
plt.imshow(output)

In [ ]:
output = ICV_transform(img_bgr, [ICV_rotate(60)])
plt.imshow(output)

In [ ]:
output = ICV_transform(img_bgr, [ICV_rotate(120)])
plt.imshow(output)

In [ ]:
output = ICV_transform(img_bgr, [ICV_rotate(-50)])
plt.imshow(output)

In [ ]:
output = ICV_transform(img_bgr, [ICV_skew(10)])
plt.imshow(output)

In [ ]:
output = ICV_transform(img_bgr, [ICV_skew(40)])
plt.imshow(output)

In [ ]:
output = ICV_transform(img_bgr, [ICV_skew(60)])
plt.imshow(output)

# 1c

In [ ]:
output = ICV_transform(img_bgr, [ICV_rotate(-20), ICV_skew(50)])
plt.imshow(output)

In [ ]:
output = ICV_transform(img_bgr, [ICV_skew(50), ICV_rotate(-20)])
plt.imshow(output)

# Convolution

# 2a

In [ ]:
def ICV_convolution(input_image, kernel):
    
    image_height, image_width = input_image.shape
    kernel_height, kernel_width = kernel.shape

    # Define the output image with the same size as the input image
    output = np.zeros_like(input_image)

    # Padding the input image to handle borders
    pad_height = kernel_height // 2
    pad_width = kernel_width // 2
    
    padded_image = np.pad(
        input_image, 
        ((pad_height, pad_height), (pad_width, pad_width)),
        mode='constant'
    )

    # Perform the convolution operation
    for i in range(image_height):
        for j in range(image_width):
            # Extract the region of interest
            region = padded_image[i:i + kernel_height, j:j + kernel_width]
            # Apply the kernel (element-wise multiplication followed by sum)
            output[i, j] = np.sum(region*kernel)

    return output



In [ ]:
# Load an example image
image = cv2.imread('./Dataset/DatasetA/car-1.jpg', 0)
plt.imshow(image, cmap='gray')

horizontal_edges = np.array([[1,0,-1],[2,0,-2],[1,0,-1]])

vertical_edges = np.array([[-1,-2,-1],[0,0,0],[1,2,1]])

kernel_a = (1/16)*np.array([[1,2,1],[2,4,2],[1,2,1]])

kernel_b = np.array([[0,1,0],[1,-4,1],[0,1,0]])

kernel_avg = np.ones((3, 3)) / 9.0

gaussian_kernel = (1/256)*np.array([
    [1 , 4  ,  6 ,  4 , 1 ],
    [4 , 16 , 24 , 16 , 4 ],
    [6 , 24 , 36 , 24 , 6 ],
    [4 , 16 , 24 , 16 , 4 ],
    [1 , 4  ,  6 ,  4 , 1 ]
])

# 2b

In [ ]:
result = ICV_convolution(image, kernel_avg)
plt.imshow(result, cmap='gray')

# 2c

In [ ]:
result = ICV_convolution(image, kernel_a)
plt.imshow(result, cmap='gray')

In [ ]:
result = ICV_convolution(image, kernel_b)
plt.imshow(result, cmap='gray')

# 2d

### (i) A followed by A

In [ ]:
result = ICV_convolution(image, kernel_a)
plt.imshow(result, cmap='gray')

In [ ]:
result2 = ICV_convolution(result, kernel_a)
plt.imshow(result2, cmap='gray')

### (ii) A followed by B

In [ ]:
result = ICV_convolution(image, kernel_a)
plt.imshow(result, cmap='gray')

In [ ]:
result2 = ICV_convolution(result, kernel_b)
plt.imshow(result2, cmap='gray')

### (iii) B followed by A

In [ ]:
result = ICV_convolution(image, kernel_b)
plt.imshow(result, cmap='gray')

In [ ]:
result2 = ICV_convolution(result, kernel_a)
plt.imshow(result2, cmap='gray')

## Video Segmentation


# 3a

In [ ]:
video = cv2.VideoCapture('./Dataset/DatasetB.avi')
fps = video.get(cv2.CAP_PROP_FPS)
print(fps)

In [ ]:
minutes = 0
seconds = 11
n_frames = int(fps*(minutes*60 + seconds))
print(n_frames)

In [ ]:
for _ in range(15):

    ret, frame = video.read()
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # plt.imshow(frame)
    # plt.show()
    
    # frequency counts of pixel intensities
    frequency_color = {
        "red":  Counter(frame[:, :, 0].flatten()),
        "green": Counter(frame[:, :, 1].flatten()),
        "blue": Counter(frame[:, :, 2].flatten())
    }

    X = list(range(255))

    fig, ax = plt.subplots()

    for k, v in frequency_color.items():
        ax.plot(X, [v[i] for i in X], label = k, color = k)
    
    # fig.savefig(f'./Dataset/DatasetB_hist_plots/color_hist_graph_{_}.png')

# 3b

In [ ]:
def histogram_intersection(hist1, hist2):
    # Ensure the histograms have the same length
    assert len(hist1) == len(hist2), "Histograms must have the same length"

    # Calculate the histogram intersection
    intersection = np.sum(np.minimum(hist1, hist2))

    return intersection

def normalize_histogram(hist):
    # Normalize the histogram to have a sum of 1
    return hist/ np.sum(hist)

def calculate_frame_intersections(video_path):
    cap = cv2.VideoCapture(video_path)

    # Read the first frame
    ret, prev_frame = cap.read()
    prev_frame_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)
    prev_hist = np.histogram(prev_frame_gray.flatten(), bins=256, range=[0, 256])[0]

    # Lists to store intersection values and normalized intersection values
    intersections = []
    normalized_intersections = []

    while True:
        ret, curr_frame = cap.read()
        if not ret:
            break

        curr_frame_gray = cv2.cvtColor(curr_frame, cv2.COLOR_BGR2GRAY)
        curr_hist = np.histogram(curr_frame_gray.flatten(), bins=256, range=[0, 256])[0]

        # Calculate histogram intersection
        intersection_value = histogram_intersection(prev_hist, curr_hist)
        intersections.append(intersection_value)

        normalized_prev_hist = normalize_histogram(prev_hist)
        normalized_curr_hist = normalize_histogram(curr_hist)
        
        # Calculate normalised histogram intersection
        normalized_intersection = histogram_intersection(normalized_prev_hist, normalized_curr_hist)
        normalized_intersections.append(normalized_intersection)

        # Update previous histogram
        prev_hist = curr_hist

    cap.release()
    return intersections, normalized_intersections

def plot_intersections(intersections, normalized_intersections):
    plt.figure(figsize=(10, 6))
    plt.plot(intersections, label='Intersection Values')
    plt.xlabel('Frame Number')
    plt.ylabel('Intersection Value')
    plt.title('Histogram Intersection Over Time')
    plt.show()

    plt.figure(figsize=(10, 6))
    plt.plot(normalized_intersections, label='Normalized Intersection Values')
    plt.xlabel('Frame Number')
    plt.ylabel('Normalized Intersection Value')
    plt.title('Normalized Histogram Intersection Over Time')
    plt.show()

# Example usage:
video_path = './Dataset/DatasetB.avi'  # Replace with the actual video path
intersections, normalized_intersections = calculate_frame_intersections(video_path)
# print(intersections, normalized_intersections)
plot_intersections(intersections, normalized_intersections)


# Texture Classification

# 4a

In [ ]:
def ICV_get_lbp_pixel(img, center, x, y): 
    new_value = 0
    try: 
        if img[x][y] >= center: 
            new_value = 1
    except IndexError: 
        pass
    return new_value 

In [ ]:
def ICV_lbp_calculated_pixel(img, x, y): 
    center = img[x][y] 
    bin_arr = [] 
    bin_arr.append(ICV_get_lbp_pixel(img, center, x-1, y-1))  # top_left 
    bin_arr.append(ICV_get_lbp_pixel(img, center, x-1, y))    # top
    bin_arr.append(ICV_get_lbp_pixel(img, center, x-1, y + 1))  # top_right
    bin_arr.append(ICV_get_lbp_pixel(img, center, x, y + 1))    # right 
    bin_arr.append(ICV_get_lbp_pixel(img, center, x + 1, y + 1))  # bottom_right
    bin_arr.append(ICV_get_lbp_pixel(img, center, x + 1, y))    # bottom
    bin_arr.append(ICV_get_lbp_pixel(img, center, x + 1, y-1))   # bottom_left
    bin_arr.append(ICV_get_lbp_pixel(img, center, x, y-1))       # left
    return int(''.join(map(str, bin_arr)), 2) 

In [ ]:
def compute_lbp_window(window):
    height, width = window.shape
    lbp_window = np.zeros((height, width), np.uint8)

    for i in range(height):
        for j in range(width):
            lbp_window[i, j] = ICV_lbp_calculated_pixel(window, i, j)

    # plot the LBP for given image window
    plt.imshow(lbp_window, cmap ="gray") 
    plt.show() 
    
    x = list(range(256))
    y = np.array([Counter(lbp_window.flatten())[i] for i in x])
    plt.bar(x, y/np.max(y)) 
    plt.show()
    return lbp_window


In [ ]:
def ICV_get_lbp_img(img, window_size):
    height, width = img.shape
    # img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_lbp = np.zeros((height, width), np.uint8)

    for i in range(0, height, window_size):
        for j in range(0, width, window_size):
            window = img[i:i+window_size, j:j+window_size]
            
            # plot the window of the image 
            plt.imshow(window, cmap ="gray") 
            plt.show() 

            lbp_window = compute_lbp_window(window)
            img_lbp[i:i+window_size, j:j+window_size] = lbp_window

   
    return img_lbp

In [ ]:
image = cv2.imread('./Dataset/DatasetA/face-3.jpg', 0)
plt.imshow(image, cmap='gray')

In [ ]:
image = cv2.imread('./Dataset/DatasetA/face-3.jpg', 0)
window_size = 64

img_lbp = ICV_get_lbp_img(image, window_size)

# Print the shape of the img_lbp array
print("Shape of img_lbp:", img_lbp.shape)

# Ensure img_lbp values are within the valid range (0 to 255)
img_lbp = np.clip(img_lbp, 0, 255).astype(np.uint8)

# Display the LBP image
plt.imshow(img_lbp, cmap='gray')
plt.show()


# 4b

In [ ]:
def ICV_lbp_classify(img, window_size):
    # Function to perform Local Binary Pattern (LBP) classification on an image
    img_height, img_width = img.shape
    hw = window_size // 2  # Half of the window size for boundary handling
    bins = 255  # Number of bins for the LBP histogram
    features = []  # List to store LBP histograms for each window
    des = np.zeros(img.shape)

    # Iterate over image windows
    for y in range(hw, img_height - hw, window_size):
        for x in range(hw, img_width - hw, window_size):

            # Extract the window from the image
            window = img[y - hw:y + hw, x - hw:x + hw]
            lbp = np.zeros(window.shape)

            # Compute LBP for each pixel in the window
            for i in range(1, window.shape[0] - 1):
                for j in range(1, window.shape[1] - 1):

                    center = window[i, j]
                    code = 0

                    # Compare the pixel values with the center pixel for LBP encoding
                    if window[i - 1, j - 1] > center: code |= 1
                    if window[i - 1, j] > center: code |= 2
                    if window[i - 1, j + 1] > center: code |= 4
                    if window[i, j + 1] > center: code |= 8
                    if window[i + 1, j + 1] > center: code |= 16
                    if window[i + 1, j] > center: code |= 32
                    if window[i + 1, j - 1] > center: code |= 64
                    if window[i, j - 1] > center: code |= 128
                    lbp[i, j] = code

            des[x:x+window_size, y:y+window_size] = lbp

            # Build LBP histogram for the window
            hist, _ = np.histogram(lbp, bins=bins, range=(0, 255))
           
            # Normalize the histogram
            hist = hist / np.max(hist)
            features.append(hist)

            # des[x, y] = lbp
            

    # plt.imshow(des, cmap='gray')
    # plt.show()

    # Sum of normalized histograms as the final feature
    normalised_sum = np.sum(features)
    return normalised_sum


In [ ]:
window_size = 64

car1 = cv2.imread('./Dataset/DatasetA/car-1.jpg',0)
# plt.imshow(car1, cmap='gray')
car2 = cv2.imread('./Dataset/DatasetA/car-2.jpg',0)
car3 = cv2.imread('./Dataset/DatasetA/car-1.jpg',0)

face1 = cv2.imread('./Dataset/DatasetA/face-1.jpg',0)
# plt.imshow(face1, cmap='gray')
face2 = cv2.imread('./Dataset/DatasetA/face-1.jpg',0)
face3 = cv2.imread('./Dataset/DatasetA/face-1.jpg',0)


car1_des_val= ICV_lbp_classify(car1, window_size)
car2_des_val = ICV_lbp_classify(car2, window_size)
car3_des_val = ICV_lbp_classify(car3, window_size)

face1_des_val= ICV_lbp_classify(face1, window_size)
face2_des_val = ICV_lbp_classify(face2, window_size)
face3_des_val = ICV_lbp_classify(face3, window_size)



print("car", car1_des_val, car2_des_val, car3_des_val)
print("face", face1_des_val, face2_des_val, face3_des_val)

# Object Counting

In [ ]:
#alternate methods: contouring

## 5a 

In [ ]:
def ICV_frame_difference(video):

    fps = video.get(cv2.CAP_PROP_FPS)
    differences=[]
    minutes = 0
    seconds = 5
    frame_id = int(fps*(minutes*60 + seconds))

    #Calculate the first frame as reference frame
    _, frame = video.read()
    referenceFrame = frame
    referenceFrame = cv2.cvtColor(referenceFrame, cv2.COLOR_BGR2GRAY)
    referenceFrame = referenceFrame.astype(int)


    for x in range(frame_id):
        _, frame = video.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        print("Selected frame")

        plt.imshow(frame, cmap='gray')
        plt.show()
        frame = frame.astype(int)
        # Compute the absolute difference between referenceFrame and frame
        difference = np.abs(referenceFrame - frame)
        print("difference")

        plt.imshow(difference, cmap='gray')
        plt.show()

        difference = np.where(difference > 40, 0, 255)
        differences.append(difference)
        
        print("threshold difference")
        plt.imshow(difference, cmap='gray')
        plt.show()
        # plt.savefig('Frame Difference By First frame '+str(x + 1)+'.jpg')

    return differences

In [ ]:
video = cv2.VideoCapture('./Dataset/DatasetC.avi')
ICV_frame_difference(video)

## 5b

In [ ]:
def ICV_frame_difference_conseq_frame(video):
    fps = video.get(cv2.CAP_PROP_FPS)
    differences=[]
    minutes = 0
    seconds = 5
    frame_id = int(fps*(minutes*60 + seconds))

    # first frame as reference frame
    _, frame = video.read()
    referenceFrame = frame
    referenceFrame = cv2.cvtColor(referenceFrame, cv2.COLOR_BGR2GRAY)
    referenceFrame = referenceFrame.astype(int)


    for x in range(frame_id):
        print(f"frame_id: {x}")
        _, frame = video.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        print("Selected frame")

        plt.imshow(frame, cmap='gray')
        plt.show()
        frame = frame.astype(int)
    
        # Compute the absolute difference between referenceFrame and frame
        difference = np.abs(referenceFrame - frame)
        print("difference")
        plt.imshow(difference, cmap='gray')
        plt.show()
        
        difference = np.where(difference > 40, 0,255)
        differences.append(difference)
        referenceFrame = frame
        # plt.figure()
        print("threshold difference")
        plt.imshow(difference, cmap='gray')
        plt.show()
        # plt.savefig('Frame Difference By previous frame '+str(x + 1)+'.jpg')


In [ ]:
video = cv2.VideoCapture('./Dataset/DatasetC.avi')
ICV_frame_difference_conseq_frame(video)

## 5c

In [ ]:
def ICV_generate_background(vid):
    it,frame = vid.read()
    
    # get the first frame
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    AllFrames = np.zeros(frame.shape, dtype='uint8')
    frameNUmber=1

    while(vid.isOpened()):
        # Reading the frames of video
        it, frame = vid.read()
        if not it:
            break

        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        # linearly increasing weights based on frame numbers
        AllFrames = (AllFrames*(frameNUmber-1)+frame)/frameNUmber
        AllFrames.astype(int)
        frameNUmber += 1
        
    pil_img=Image.fromarray((AllFrames).astype(np.uint8)) 
    plt.imshow(pil_img,cmap='gray')
    pil_img.save('Background.jpg')

In [ ]:
video = cv2.VideoCapture('./Dataset/DatasetC.avi')
ICV_generate_background(video)

## 5d

In [ ]:
def ICV_object_counter(global_backgroud_image, video):
    object_counts = []

    for x in range(125):
        _, frame = video.read()
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
     
        # Compute the absolute difference between referenceFrame and frame
        difference = np.abs(global_backgroud_image - frame)

        # set manual threshold for difference
        difference = np.where(difference > 100, 255, 0)
        object_count = np.count_nonzero(difference) // 8000
        
        object_counts.append(object_count)
    
    return object_counts

In [ ]:
video = cv2.VideoCapture('./Dataset/DatasetC.avi')
global_backgroud_image = cv2.imread("Background.jpg",0)
objects_count = ICV_object_counter(global_backgroud_image,video)
print(objects_count)

In [ ]:
plt.bar(list(range(125)), objects_count)
plt.xlabel('Video frame')
plt.ylabel('objects')
plt.title('Ojects per frame')
plt.show()